## Comparison of analytic IK module against real robot

we collected pairs of joint configs and eef poses for a real UR5e. We included challenging poses such as ones that are on the edge of the workspace or close to a singularity. 

In this notebook we compare calculated IK/FK against the actual joint configs/poses to see 
1. if the code works
2. assess the impact of the default DH params vs the calibrated DH params
3. assert the completeness of the IK equations




In [29]:
import numpy as np
from ur_analytic_ik import ur5e

np.set_printoptions(precision=4, suppress=True) # Prettier printing


In [30]:
with open("ur5e_poses.txt") as f:
    lines = f.readlines()


In [31]:
joint_config_pose_pairs = []
for line in lines:
    if len(line) == 1:
        continue
    joint_config, pose = line.split(";")[0], line.split(";")[1]
    joint_config = joint_config.split(",")
    # strip the brackets
    joint_config[0] = joint_config[0][1:]
    joint_config[-1] = joint_config[-1][:-1]
    joint_config = [float(j) for j in joint_config]
    
    pose = pose.split(",")
    # remove all brackets
    for i in range(len(pose)):
        pose[i] = pose[i].replace("[", "").replace("]", "")
    pose = [float(p) for p in pose]
    pose = np.array(pose).reshape(4, 4)
    joint_config_pose_pairs.append((joint_config, pose))


In [32]:
fk_poses = []

for joint_config, pose in joint_config_pose_pairs:
    fk_pose = ur5e.forward_kinematics(*joint_config)
    fk_poses.append(fk_pose)
    distance = np.linalg.norm(fk_pose[:3,3] - pose[:3,3])
    print(f"fk pose vs real pose with distance {distance} m")



fk pose vs real pose with distance 0.0018118633601494816 m
fk pose vs real pose with distance 0.001244811756654254 m
fk pose vs real pose with distance 0.0016960076410719273 m
fk pose vs real pose with distance 0.001823231630988334 m
fk pose vs real pose with distance 0.0013480485905176857 m
fk pose vs real pose with distance 0.001334366907552316 m
fk pose vs real pose with distance 0.001429664226362876 m
fk pose vs real pose with distance 0.0014651598985280685 m
fk pose vs real pose with distance 0.0014481558396959072 m
fk pose vs real pose with distance 0.0018434954707430456 m


we can see that the robot's poses differ approx 2mm from the calculated poses, most likely due to the calibrated DH parameters.

In [33]:
for joint_config,pose in joint_config_pose_pairs:
    joint_config_ik = ur5e.inverse_kinematics_closest(pose,*joint_config)
    if len(joint_config_ik) == 0:
        print("no ik solution found")
        continue
    joint_config_ik = np.array(joint_config_ik[0])

    distance = np.linalg.norm(joint_config-joint_config_ik)
    print(f"joint config vs ik joint config with distance {distance} rad")

joint config vs ik joint config with distance 0.007135091262535225 rad
joint config vs ik joint config with distance 0.0068736539252271835 rad
no ik solution found
joint config vs ik joint config with distance 0.00648563672138501 rad
joint config vs ik joint config with distance 0.011152488236170929 rad
joint config vs ik joint config with distance 0.01071535318145833 rad
joint config vs ik joint config with distance 0.008245401296536689 rad
joint config vs ik joint config with distance 0.008531492436559675 rad
joint config vs ik joint config with distance 0.00854205339004058 rad
joint config vs ik joint config with distance 0.007467297721403651 rad


one of the poses has no IK solution, which might seem problematic. However, this is a pose where the robot is fully stretched out. Moving it 1mm towards the base, results in an IK solution, as can be seen below:

Two plausible causes:

1. different between actual DH params and default DH params (could be indeed unreachable for default)
2. limit in coverage of the analytic IK on the boundary of the workspace, where you never want to be anyways.

In [41]:
joints, pose = joint_config_pose_pairs[2]

pose_test = pose.copy()
pose_test[2,3] -= 0.001

joints_ik = ur5e.inverse_kinematics_closest(pose_test, *joints)
print(joints_ik)
print(joints)
print(np.linalg.norm(joints-joints_ik[0]))

[array([-0.011 , -1.58  ,  0.0171, -0.0061,  1.5723,  0.0049])]
[1.430511474609375e-06, -1.5707653678221796, -5.975520616630092e-05, -5.525401611805592e-07, 1.5707817077636719, -1.1746083394825746e-05]
0.02371973657362765
